<h1>Auto-avaliação e Formação de grupos</h1>

Importando as bibliotecas que iremos utilizar ao longo de todo o código.

In [1]:
import pandas as pd
from itertools import combinations

<h2>1. Agrupamento por Personalidade (Myers-Briggs)

<h3>1.1. Leitura da Base de Dados

Lendo o Dataframe com os "eixos" do Teste Myers-Briggs.

In [2]:
df_eixo = pd.read_excel("Base_Dados.xlsx", sheet_name = 'Eixo')

#salvando a coluna de nomes para mais tarde, 
#e criando um dicionário que associe o nome com o índice da linha
nomes = df_eixo['ID']
nomes_dict = dict(enumerate(nomes))

df_eixo.set_index(df_eixo["ID"], inplace=True)
df_eixo.drop(['ID'], axis=1)

#O comando Head mostra só as primeiras 5 linhas da minha base
df_eixo.head()

,ID,Mente,Energia,Natureza,Táticas,Identidade
ID,,,,,,
Gustavo,Gustavo,0.490000,0.730000,0.430000,0.710000,0.260000
João,João,0.829325,0.735402,0.629517,0.052167,0.175216
Maria,Maria,0.116569,0.893693,0.624196,0.742562,0.163249
José,José,0.657260,0.074111,0.437428,0.602109,0.149548
Roberto,Roberto,0.898011,0.056593,0.025091,0.537110,0.493394


**Obs.** É necessário inputar somente o primeiro valor de caida eixo, o segundo vem por complemento.

**Exemplo:** No eixo mente, consideramos somente o valor de extrovertido (49%), o valor de introvertido (51%) vem por complemento.





Lendo o Dataframe com todos os valores

In [3]:
df_valores = pd.read_excel("Base_Dados.xlsx", sheet_name = 'Valores')
df_valores.set_index(df_eixo["ID"], inplace=True)
df_valores.drop(["ID"],axis=1,inplace=True)
df_valores.head()

,Extrovertido,Introvertido,Intuitivo,Observador,Pensante,Sentimento,Julgador,Explorador,Assertivo,Cauteloso
ID,,,,,,,,,,
Gustavo,0.490000,0.510000,0.730000,0.270000,0.430000,0.570000,0.710000,0.290000,0.260000,0.740000
João,0.829325,0.170675,0.735402,0.264598,0.629517,0.370483,0.052167,0.947833,0.175216,0.824784
Maria,0.116569,0.883431,0.893693,0.106307,0.624196,0.375804,0.742562,0.257438,0.163249,0.836751
José,0.657260,0.342740,0.074111,0.925889,0.437428,0.562572,0.602109,0.397891,0.149548,0.850452
Roberto,0.898011,0.101989,0.056593,0.943407,0.025091,0.974909,0.537110,0.462890,0.493394,0.506606


Lendo o DataFrame que contém a sigla do tipo de personalidade:

In [4]:
df_personalidade = pd.read_excel("Base_Dados.xlsx", sheet_name = "Sigla")
df_personalidade.set_index(df_personalidade["ID"], inplace=True)
df_personalidade.drop(["ID"],axis=1,inplace=True)
df_personalidade.head()

,Sigla
ID,
Gustavo,INFP
João,ENTJ
Maria,INFP
José,ESFP
Roberto,ESFP


<h3>1.2. Formando Pares: Um Teste

Encontrei algumas técnicas de agrupamento. Resolvi priorizar o somatório das polaridades dos eixos. Pessoas mais equilibradas (que estão posicionadas no meio do eixo) terão um valor baixo, enquanto pessoas que estão nas extremidas dos eixos terão um valor (positivo ou negativo) mais alto.

Fonte: https://thoughtcatalog.com/heidi-priebe/2017/02/8-ways-of-grouping-myers-briggs-types-that-would-make-more-sense-than-keirseys-temperament-groups/

Invertendo o sinal das variáveis mais à esquerda de cada eixo, para ser capaz de se chegar a uma métrica com polaridade:

In [5]:
df_valores["Extrovertido"] = - df_valores["Extrovertido"]
df_valores["Intuitivo"] = - df_valores["Intuitivo"]
df_valores["Pensante"] = - df_valores["Pensante"]
df_valores["Julgador"] = - df_valores["Julgador"] 
df_valores["Assertivo"] = -df_valores["Assertivo"]
df_valores.head()

,Extrovertido,Introvertido,Intuitivo,Observador,Pensante,Sentimento,Julgador,Explorador,Assertivo,Cauteloso
ID,,,,,,,,,,
Gustavo,-0.490000,0.510000,-0.730000,0.270000,-0.430000,0.570000,-0.710000,0.290000,-0.260000,0.740000
João,-0.829325,0.170675,-0.735402,0.264598,-0.629517,0.370483,-0.052167,0.947833,-0.175216,0.824784
Maria,-0.116569,0.883431,-0.893693,0.106307,-0.624196,0.375804,-0.742562,0.257438,-0.163249,0.836751
José,-0.657260,0.342740,-0.074111,0.925889,-0.437428,0.562572,-0.602109,0.397891,-0.149548,0.850452
Roberto,-0.898011,0.101989,-0.056593,0.943407,-0.025091,0.974909,-0.537110,0.462890,-0.493394,0.506606


Calculando as diferenças entre as duas variáveis de cada um dos eixos para ver o grau de polarização das pessoas

In [6]:
df_dif_eixo = pd.DataFrame()
df_dif_eixo["Mente"] = df_valores["Extrovertido"] + df_valores["Introvertido"]
df_dif_eixo["Energia"] = df_valores["Intuitivo"] + df_valores["Observador"]
df_dif_eixo["Natureza"] = df_valores["Pensante"] + df_valores["Sentimento"]
df_dif_eixo["Táticas"] = df_valores["Julgador"] + df_valores["Explorador"]
df_dif_eixo["Identidade"] = df_valores["Assertivo"] + df_valores["Cauteloso"]

df_dif_eixo

,Mente,Energia,Natureza,Táticas,Identidade
ID,,,,,
Gustavo,0.020000,-0.460000,0.140000,-0.420000,0.480000
João,-0.658649,-0.470804,-0.259033,0.895666,0.649569
Maria,0.766861,-0.787387,-0.248392,-0.485125,0.673502
José,-0.314520,0.851777,0.125145,-0.204218,0.700904
Roberto,-0.796022,0.886815,0.949818,-0.074220,0.013212
Lucas,-0.665934,-0.138207,-0.576789,-0.257782,0.953010
Pedro,0.299000,-0.225711,0.716039,0.750641,-0.349516
Álvaro,0.766926,0.326389,-0.843162,-0.311919,0.476091
Charles,-0.421811,0.914540,-0.677702,-0.380717,0.213429


Agora temos que lidar com todas as combinações possíveis de pessoas, em cada um dos eixos para tentar maximizar as diferenças existentes.

In [7]:
#Calculando as combinações para dois casos

index_combinations = pd.Series(combinations(range(len(df_dif_eixo)),2))
column_names = ["Combinação","Mente","Energia", "Natureza", "Táticas", "Identidade"]

s_mente = combinations(df_dif_eixo["Mente"], 2)
s_energia = combinations(df_dif_eixo["Energia"], 2)
s_natureza = combinations(df_dif_eixo["Natureza"], 2)
s_taticas = combinations(df_dif_eixo["Táticas"], 2)
s_identidade = combinations(df_dif_eixo["Identidade"], 2)

In [8]:
#Fazendo uma função para calcular a amplitude de cada tupla
def dif_combinacoes(serie):
    return [x[1] - x[0] for x in serie]

In [9]:
#Somando os valores das combinações
s_mente = pd.Series(dif_combinacoes(s_mente))
s_energia = pd.Series(dif_combinacoes(s_energia))
s_natureza = pd.Series(dif_combinacoes(s_natureza))
s_taticas = pd.Series(dif_combinacoes(s_taticas))
s_identidade = pd.Series(dif_combinacoes(s_identidade))

In [10]:
#Criando um dataframe para as combinações
df_combinations = pd.DataFrame()
df_combinations = pd.concat([index_combinations, s_mente,s_energia,s_natureza,s_taticas,s_identidade],axis=1)
df_combinations.columns = column_names
df_combinations['C0'] = df_combinations["Combinação"].map(lambda x:x[0])
df_combinations['C1'] = df_combinations["Combinação"].map(lambda x:x[1])

#Substituindo o índice por nomes
df_combinations['C0'] = df_combinations['C0'].map(lambda x:nomes[x])
df_combinations['C1'] = df_combinations['C1'].map(lambda x:nomes[x])

In [11]:
#Avaliando e ordenando os melhroes Scores de diversidade das Combinações
df_combinations["SCORE"] = df_combinations.sum(axis=1)
df_combinations.head()

,Combinação,Mente,Energia,Natureza,Táticas,Identidade,C0,C1,SCORE
0,"(0, 1)",-0.678649,-0.010804,-0.399033,1.315666,0.169569,Gustavo,João,0.396747
1,"(0, 2)",0.746861,-0.327387,-0.388392,-0.065125,0.193502,Gustavo,Maria,0.159460
2,"(0, 3)",-0.334520,1.311777,-0.014855,0.215782,0.220904,Gustavo,José,1.399088
3,"(0, 4)",-0.816022,1.346815,0.809818,0.345780,-0.466788,Gustavo,Roberto,1.219603
4,"(0, 5)",-0.685934,0.321793,-0.716789,0.162218,0.473010,Gustavo,Lucas,-0.445703


In [12]:
#Extraindo o módulo (o sinal da polaridade já foi considerado no cálculo da amplitude)
#e organizando o DF em ordem descendente

df_combinations["SCORE"] = df_combinations["SCORE"].map(lambda x: abs(x))
df_combinations = df_combinations.sort_values("SCORE", ascending=False)
df_combinations.head()

,Combinação,Mente,Energia,Natureza,Táticas,Identidade,C0,C1,SCORE
40,"(5, 6)",0.964934,-0.087504,1.292828,1.008424,-1.302526,Lucas,Pedro,1.876156
28,"(3, 5)",-0.351414,-0.989984,-0.701933,-0.053564,0.252105,José,Lucas,1.844790
43,"(5, 9)",-0.230839,0.721216,0.252648,0.988113,-0.025183,Lucas,Paulo,1.705956
34,"(4, 5)",0.130087,-1.025022,-1.526607,-0.183562,0.939798,Roberto,Lucas,1.665306
44,"(5, 10)",1.116222,-0.446259,0.571853,0.913870,-0.572290,Lucas,Mariazinha,1.583396


É a partir desta tabela com os SCORES que formaremos os grupos

In [13]:
#Função Forma Pares

def forma_pares(df_sorted):
    
    """
    input -> Dataframe Organizado por Score
    output -> Lista de Pares
    
    Funcionamento:
    Organiza pares procurando otimizar o score de diversidade 
    """
    
    lista_pares = []
    
    while df_sorted.empty == False:
        
        par = df_sorted.iloc[0]["Combinação"]

        #Seleciona as pessoas do par
        p1 = nomes[par[0]]
        p2 = nomes[par[1]]

        #Adiciona o primeiro par na lista geral de pares
        lista_pares.append((p1,p2))

        #Elimina este par para ele não ser sorteado novamente    
        df_sorted.drop([0],axis=0,inplace=True)

        #Elimina todas as outras combinações contendo os participantes dos pares, por meio de boolean mask
        bool_mask = ((df_sorted['C0'] == p1) | (df_sorted['C1'] == p2) | (df_sorted['C0'] == p2) | (df_sorted['C1'] == p1))
        df_sorted = df_sorted[~bool_mask]

        #Reseta os índices do Dataframe
        df_sorted.reset_index(drop=True, inplace=True)

    todos_com_par = list(zip(*lista_pares))
    todos_com_par = todos_com_par[0] + todos_com_par[1]
    
    for nome in nomes:
        if nome not in todos_com_par:
            lista_pares.append(nome)
    
    return lista_pares

pares = forma_pares(df_combinations)
print(pares)

[('Lucas', 'Pedro'), ('José', 'Charles'), ('Gustavo', 'Paulo'), ('Maria', 'Roberto'), ('João', 'Mariazinha'), 'Álvaro']


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


<h3>1.3. Extrapolando a formação de pares para grupos com N integrantes

In [14]:
#Criando uma função para formar grupos com n_integrantes

def combina_eixos(df,n_grupo):
    
    '''
    Realiza todas as combinações possíveis entre os eixos de um grupo com n integrantes
    '''
    
    column_names = ["Mente","Energia", "Natureza", "Táticas", "Identidade"]
    df_comb = pd.DataFrame()
    df_comb["Combinação"] = pd.Series(combinations(range(len(df)),n_grupo))

    for name in column_names:
        df_comb[name] = pd.Series(combinations(df[name],n_grupo))

    return df_comb

df2 = combina_eixos(df_dif_eixo,3)
df2.head()

,Combinação,Mente,Energia,Natureza,Táticas,Identidade
0,"(0, 1, 2)","(0.020000000000000018, -0.6586490466057533, 0....","(-0.45999999999999996, -0.4708044415031136, -0...","(0.14000000000000007, -0.2590333236108291, -0....","(-0.41999999999999993, 0.8956656413660975, -0....","(0.48, 0.6495685209897342, 0.6735021400016254)"
1,"(0, 1, 3)","(0.020000000000000018, -0.6586490466057533, -0...","(-0.45999999999999996, -0.4708044415031136, 0....","(0.14000000000000007, -0.2590333236108291, 0.1...","(-0.41999999999999993, 0.8956656413660975, -0....","(0.48, 0.6495685209897342, 0.7009041908997489)"
2,"(0, 1, 4)","(0.020000000000000018, -0.6586490466057533, -0...","(-0.45999999999999996, -0.4708044415031136, 0....","(0.14000000000000007, -0.2590333236108291, 0.9...","(-0.41999999999999993, 0.8956656413660975, -0....","(0.48, 0.6495685209897342, 0.013211606866609493)"
3,"(0, 1, 5)","(0.020000000000000018, -0.6586490466057533, -0...","(-0.45999999999999996, -0.4708044415031136, -0...","(0.14000000000000007, -0.2590333236108291, -0....","(-0.41999999999999993, 0.8956656413660975, -0....","(0.48, 0.6495685209897342, 0.953009676736803)"
4,"(0, 1, 6)","(0.020000000000000018, -0.6586490466057533, 0....","(-0.45999999999999996, -0.4708044415031136, -0...","(0.14000000000000007, -0.2590333236108291, 0.7...","(-0.41999999999999993, 0.8956656413660975, 0.7...","(0.48, 0.6495685209897342, -0.3495158505080014)"


In [15]:
def delta_combinacao(df,n_grupo):
    
    '''
    Calcula o delta máximo em um dataframe saído da função combina_eixos()  
    Obs. Lembre-se que cada linha é uma combinação de três pessoas
    
    '''
    names = ["Mente","Energia", "Natureza", "Táticas", "Identidade"]
    
    df_delta = pd.DataFrame()

    for name in names:
        df_delta[name] = df[name].apply(lambda x: max(x)-min(x))
    
    df_delta["SCORE"] = df_delta.sum(axis=1)
    df_delta["Combinação"] = df["Combinação"]
    
    return df_delta

df2 = delta_combinacao(df2,3)
df2.head()

,Mente,Energia,Natureza,Táticas,Identidade,SCORE,Combinação
0,1.425510,0.327387,0.399033,1.380790,0.193502,3.726223,"(0, 1, 2)"
1,0.678649,1.322582,0.399033,1.315666,0.220904,3.936834,"(0, 1, 3)"
2,0.816022,1.357619,1.208852,1.315666,0.636357,5.334515,"(0, 1, 4)"
3,0.685934,0.332597,0.716789,1.315666,0.473010,3.523995,"(0, 1, 5)"
4,0.957649,0.245093,0.975073,1.315666,0.999084,4.492565,"(0, 1, 6)"


In [16]:
def sort_on_Score(df):
    
    '''Organiza um Dataframe em ordem decrescente a partir da coluna Score'''
    
    return df.sort_values("SCORE", ascending=False)

In [17]:
def flatten(t):
    '''Função que retorna uma lista flat a partir de uma lista de listas'''
    return [item for sublist in t for item in sublist]

In [18]:
def rename_combinations(df,n_grupo):
    
    '''Cria novas colunas no DF substituindo os números das tuplas 
    na coluna Combinação pelos nomes dos integrantes do grupo nas 
    colunas C0, C1, C2, etc.'''
    
    labels = ["C%d"%n for n in range(n_grupo)]
    comb_df = pd.DataFrame(df['Combinação'].tolist(),columns=labels)
    df = df.join(comb_df)
    
    for label in labels:
        df[label] = df[label].apply(lambda x:nomes_dict[x])
        
    return df

df3 = rename_combinations(df2,3)
df3 = sort_on_Score(df3)
df3.head()

,Mente,Energia,Natureza,Táticas,Identidade,SCORE,Combinação,C0,C1,C2
92,1.663635,1.674202,1.273959,1.215456,0.914615,6.741867,"(2, 4, 9)",Maria,Roberto,Paulo
89,1.562883,1.674202,1.198210,1.235766,1.023018,6.694079,"(2, 4, 6)",Maria,Roberto,Pedro
101,1.663635,1.370396,1.040180,1.235766,1.277343,6.587319,"(2, 6, 9)",Maria,Pedro,Paulo
62,1.562947,1.357619,1.792981,1.207584,0.636357,6.557489,"(1, 4, 7)",João,Roberto,Álvaro
100,1.188672,1.701926,1.393742,1.235766,1.023018,6.543124,"(2, 6, 8)",Maria,Pedro,Charles


In [19]:
#Função para Formar Grupos

def forma_grupos(df_sorted):
    
    lista_grupos = []

    while df_sorted.empty == False:
        
        tamanho_grupo = len(df_sorted.iloc[0]["Combinação"])
        colunas = ["C%d"%n for n in range(tamanho_grupo)]

        #Seleciona as pessoas do grupo
        pessoas_grupo = []
        
        for coluna in colunas:
            pessoas_grupo.append(df_sorted.iloc[0][coluna])

        #Adiciona o primeiro grupo na lista geral de pares
        lista_grupos.append(pessoas_grupo)

        #Elimina este grupo para ele não ser sorteado novamente    
        df_sorted.drop([0],axis=0,inplace=True)

        #Elimina todas as outras combinações contendo os participantes dos grupos, por meio de boolean mask
        bool_mask = False
        for pessoa in pessoas_grupo:
            for coluna in colunas:
                bool_mask = bool_mask | (df_sorted[coluna] == pessoa)
                
        df_sorted = df_sorted[~bool_mask]

        #Reseta os índices do Dataframe
        df_sorted.reset_index(drop=True, inplace=True)
    
    #Depois de agrupar todo mundo, mostra quem sobrou
    for nome in nomes:
        if nome not in flatten(lista_grupos):
            lista_grupos.append(nome)
    
    return lista_grupos

grupos = forma_grupos(df3)
grupos

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


[['Maria', 'Roberto', 'Paulo'],
 ['João', 'Pedro', 'Charles'],
 ['José', 'Álvaro', 'Mariazinha'],
 'Gustavo',
 'Lucas']